In [1]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import os
import re
import pandas as pd
import random
pd.set_option('display.max_colwidth', None)

In [2]:
def preprocessing(text):
    
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace
    return ascii_text

In [3]:
def statut_extraction_tabac(doc):
    statut = "UNKNOWN"
    n_oui = 0
    n_non = 0
    value_is_negated = False
    value_other_detected = False
    
    for ann in doc.anns:
        for attr in ann.attrs:
            if ann.label == "tabagisme":
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value
                if value_other_detected:
                    continue
                else:
                    if value_is_negated:
                        n_non += 1
                    else:
                        n_oui += 1
                break  # On sort de la boucle après avoir récupéré la première valeur

    if n_non > 0 and n_oui > 0:
        statut = "FUMEUR"
    elif n_non > 0:
        statut = "NON-FUMEUR"
    elif n_oui > 0:
        statut = "FUMEUR"
    return statut

In [4]:
def statut_extraction_alcool(doc):
    statut = "UNKNOWN"
    n_oui = 0
    n_non = 0
    value_is_negated = False
    value_other_detected = False
    count = 0  # Variable de compteur pour suivre le nombre d'occurrences

    for ann in doc.anns:
        for attr in ann.attrs:
            if ann.label == "alcool":
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value
                count += 1  # Incrémenter le compteur

                if count == 2:  # Vérifier si le compteur atteint 2
                    break  # Sortir de la boucle interne une fois que la deuxième valeur est récupérée

        if count == 2:  # Sortir de la boucle externe une fois que la deuxième valeur est récupérée
            if value_other_detected:
                continue
            else:
                if value_is_negated:
                    n_non += 1
                else:
                    n_oui += 1

            if n_non > 0 and n_oui > 0:
                statut = "ALCOOLIQUE"
            elif n_non > 0:
                statut = "NON-ALCOOLIQUE"
            elif n_oui > 0:
                statut = "ALCOOLIQUE"
            break

    return statut

In [5]:
def statut_extraction_situation_familiale(doc):
    ## Initialisation
    situation = "UNKNOWN"
    value_is_negated = False
    value_other_detected = False
    count = 0  # Variable de compteur pour suivre le nombre d'occurrences

    # On parcourt le dico pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "situation":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                situation = ann.text.lower()
                ## NORMALISATION: Seul, pas seul ou inconnu
                if re.search(r"\bmarie[e]?\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\bcelibataire\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bdivorce[e]?\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bveuf\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bveuve\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bpacse[e][s]?\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\bconcubinage\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\b(vit|habite)\sseul(e)?\b", situation):
                    situation = "SEUL"

                # Si il y a une négation
                if value_is_negated == True:
                    # On inverse statut_marital
                    if situation == "SEUL":
                        situation = "PAS SEUL"
                    else:
                        situation = "SEUL"

                if count == 3:  # Sortir de la boucle externe une fois que la troisième valeur est récupérée
                    break
    return situation  

In [6]:
def clinical_case_recovery(output_folder,option_melange):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]
    
    if option_melange == True:
        # On mélange les fichiers de manière aléatoire
        random.shuffle(txt_files)
    else:
        # On trie les fichiers par ordre alphabétique
        txt_files = sorted(txt_files)

    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(len(txt_files)):
        file_path = os.path.join(output_folder, txt_files[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files[i]] = text
    return textes

## LES NEG DETECTOR

In [7]:

def neg_detector_tabac():

    ### QUELQUES REGEX NEGATION

    neg_rules = [       

    NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend|fume)\s*pas"),
    NegationDetectorRule(regexp=r"jamais"),
    NegationDetectorRule(regexp=r"\bni\b"),
    NegationDetectorRule(regexp=r"\bnon\s+\b"),
    NegationDetectorRule(regexp=r"Tabac\s*[=:]?\s*0"),
    NegationDetectorRule(regexp=r"(pas|ni|ou)\s+de\s+(consommation\s+de\s+)?taba"),
    NegationDetectorRule(regexp=r"pas\s+d\'intoxication\s+tabagi"),
    NegationDetectorRule(regexp=r"0 tabac"),

    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [8]:
def neg_detector_alcool():

    ### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
        NegationDetectorRule(regexp=r"\bpas\b"),
        NegationDetectorRule(regexp=r"\becarte\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [9]:
def neg_detector_situation_familiale():

### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

## REGEX

In [10]:
regexp_rules_tabac = [
    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?\b", label="tabagisme", exclusion_regexp = "residu(s)?/s+de/s+fumee(s)?"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumeur|fumeuse", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumait", label="tabagisme"),

]
regexp_rules_alcool = [
    RegexpMatcherRule(regexp=r"\balcool(o)?\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\bbuvait\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="alcool"),
]

regexp_rules_familial = [
    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="situation"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="situation"),
]

## EXTRACTION STATUTS

In [11]:
def extraction_finale(clinical_case_repo,option_melange):
    # Listes pour stocker les données
    data_tabac = []
    data_alcool = []
    data_situation = []
    data = []
    docs = []
    # On charge les cas cliniques dans un dico {nom fichier: cas clinique}
    clinical_cases_dico = clinical_case_recovery(clinical_case_repo,option_melange)

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case) # prétraitement du texte
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)

        # Création de l'objet neg detector
        neg_detector_tabac_obj = neg_detector_tabac()
        neg_detector_alcool_obj = neg_detector_alcool()
        neg_detector_statut_familial_obj = neg_detector_situation_familiale()

        # On applique neg detector aux syntagmas
        neg_detector_tabac_obj.run(syntagmas)
        neg_detector_alcool_obj.run(syntagmas)
        neg_detector_statut_familial_obj.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_familial = RegexpMatcher(rules=regexp_rules_familial, attrs_to_copy=["is_negated", "other_detected"])

        entities_tabac = regexp_matcher_tabac.run(syntagmas)
        entities_alcool = regexp_matcher_alcool.run(syntagmas)
        entities_familial = regexp_matcher_familial.run(syntagmas)

        for entity in entities_tabac:
            doc.anns.add((entity))
           
        for entity in entities_alcool:
            doc.anns.add((entity))
        
        for entity in entities_familial:
            doc.anns.add((entity))
        
        docs.append(doc)
               
        tabagisme = statut_extraction_tabac(doc)
        alcool = statut_extraction_alcool(doc)
        situation = statut_extraction_situation_familiale(doc)

        # Remplissage de data
        data_tabac.append([fichier, clinical_case, tabagisme])
        data_alcool.append([fichier, clinical_case, alcool])
        data_situation.append([fichier, clinical_case, situation])
        data.append([fichier, clinical_case, tabagisme,alcool,situation])

    df_tabac = pd.DataFrame(data_tabac, columns=["nom fichier", "cas clinique", "tabagisme"])
    df_alcool = pd.DataFrame(data_alcool, columns=["nom fichier", "cas clinique", "alcool"])
    df_situation = pd.DataFrame(data_situation, columns=["nom fichier", "cas clinique", "situation"])
    df = pd.DataFrame(data, columns=["nom fichier", "cas clinique", "tabagisme", "alcool", "situation"])
    
    return df,docs

In [12]:
df,docs = extraction_finale("clinical_case2",option_melange=True)
print("fin")

fin


# PREPARATION DES FICHIERS A ANNOTER 

In [13]:
df['alcool'].value_counts()

alcool
UNKNOWN           685
ALCOOLIQUE         22
NON-ALCOOLIQUE     10
Name: count, dtype: int64

In [14]:
df['situation'].value_counts()

situation
UNKNOWN     696
PAS SEUL     11
SEUL         10
Name: count, dtype: int64

## PREPARATION ANNOTATION TABAC

In [15]:
df['tabagisme'].value_counts()

tabagisme
UNKNOWN       665
FUMEUR         37
NON-FUMEUR     15
Name: count, dtype: int64

In [16]:
df_non_fumeur = df[df["tabagisme"] == "NON-FUMEUR"]
df_fumeur = df[df["tabagisme"] == "FUMEUR"].head(19)
df_unknown = df[df["tabagisme"] == "UNKNOWN"].head(17)

In [17]:
df_statut_tabac = pd.concat([df_non_fumeur, df_fumeur, df_unknown], ignore_index=True)

In [18]:
df_statut_tabac.shape

(51, 5)

In [196]:
# Récupération dans un dossier des fichiers textes à annoter
import os
import shutil

chemin_dossier_source = "clinical_case2"
#chemin_dossier_destination = "51_fichiers_annotation_tabac"
# On crée le dossier destination si il n'existe pas 
os.makedirs(chemin_dossier_destination, exist_ok=True)

for nom_fichier in df_statut_tabac['nom fichier']:
    chemin_fichier_source = os.path.join(chemin_dossier_source, nom_fichier)
    chemin_fichier_destination = os.path.join(chemin_dossier_destination, nom_fichier)
    shutil.copy(chemin_fichier_source, chemin_fichier_destination)

In [117]:
# On tri le 
df_statut_tabac = df_statut_tabac.sort_values('nom fichier')

In [118]:
df_statut_tabac

nom fichier   
1     filehtml-23-cas.txt  \
7     filehtml-24-cas.txt   
11    filehtml-31-cas.txt   
42    filehtml-34-cas.txt   
32  filepdf-103-1-cas.txt   
38    filepdf-119-cas.txt   
44    filepdf-121-cas.txt   
33    filepdf-130-cas.txt   
43    filepdf-163-cas.txt   
45    filepdf-167-cas.txt   
39    filepdf-170-cas.txt   
49  filepdf-171-1-cas.txt   
16    filepdf-193-cas.txt   
13    filepdf-221-cas.txt   
48    filepdf-231-cas.txt   
27  filepdf-263-1-cas.txt   
18  filepdf-263-2-cas.txt   
24  filepdf-263-3-cas.txt   
22    filepdf-273-cas.txt   
0      filepdf-28-cas.txt   
17    filepdf-454-cas.txt   
19    filepdf-474-cas.txt   
25     filepdf-50-cas.txt   
15     filepdf-52-cas.txt   
47  filepdf-533-3-cas.txt   
41  filepdf-534-5-cas.txt   
28  filepdf-630-2-cas.txt   
35  filepdf-630-5-cas.txt   
10    filepdf-680-cas.txt   
23     filepdf-69-cas.txt   
3     filepdf-710-cas.txt   
12    filepdf-743-cas.txt   
5     filepdf-750-cas.txt   
46    filepdf-751-cas.txt   
6     filepdf-756-cas.txt   
4     filepdf-757-cas.txt   
36   filepdf-79-2-cas.txt   
37    filepdf-792-cas.txt   
40  filepdf-800-3-cas.txt   
34    filepdf-815-cas.txt   
21    filepdf-830-cas.txt   
8     filepdf-836-cas.txt   
9   filepdf-840-1-cas.txt   
14    filepdf-843-cas.txt   
29  filepdf-847-1-cas.txt   
50    filepdf-854-cas.txt   
20    filepdf-856-cas.txt   
30    filepdf-857-cas.txt   
26     filepdf-88-cas.txt   
2   filepdf-883-1-cas.txt   
31     filepdf-96-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

## On fait tourner le code sur ces fichiers uniquement

In [59]:
df,docs_medkit = extraction_finale("51_fichiers_annotation_tabac",option_melange=False)
print("fin")

fin


## EVALUATION: COMPARAISON ENTRE ANNOTATION BRAT ET MEDKIT

In [60]:
# On récupère les annotations brat

from medkit.io.brat import BratInputConverter

# Define Input Converter 
brat_converter = BratInputConverter()

path= "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_tabac"

# Load brat into a list of documents
docs_brat = brat_converter.load(dir_path = path)
len(docs_brat)

51

In [61]:
from medkit.core.text import Span
from medkit.core.text import Entity

def convert_to_pred_ents(docs_medkit):
    pred_ents = []
    
    for doc in docs_medkit:
        entities = []
        for entity in doc.anns:
            entity_spans = [Span(start=span.start, end=span.end) for span in entity.spans]
            entity_obj = Entity(label=entity.label, spans=entity_spans, text=entity.text)
            entities.append(entity_obj)
        
        pred_ents.append(entities)
    
    return pred_ents

In [62]:
pred_ents = convert_to_pred_ents(docs_medkit)
pred = convert_to_pred_ents(docs_brat)

In [63]:
pred_ents[15:20]

[[Entity(uid='ac28087e-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac28087e-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=65, end=74)], text='tabagique')],
 [Entity(uid='ac28087f-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac28087f-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=30, end=39)], text='tabagique')],
 [Entity(uid='ac280880-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac280880-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=50, end=59)], text='tabagisme')],
 [Entity(uid='ac280881-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac280881-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=80, end=89)], text='tabagique')],
 [Entity(uid='ac280882-197d-11ee-875d-b916f667b0

In [64]:
pred[15:20]

[[Entity(uid='ac2808b2-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac2808b2-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=65, end=74)], text='tabagique')],
 [Entity(uid='ac2808b3-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac2808b3-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=30, end=39)], text='tabagique')],
 [Entity(uid='ac2808b4-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac2808b4-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=50, end=59)], text='tabagisme')],
 [Entity(uid='ac2808b5-197d-11ee-875d-b916f667b06e', label='tabagisme', attrs=EntityAttributeContainer(ann_id='ac2808b5-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=80, end=89)], text='tabagique')],
 [Entity(uid='ac2808b6-197d-11ee-875d-b916f667b0

In [65]:
from medkit.core.text import TextDocument, Entity, Span
from medkit.text.metrics.ner import SeqEvalEvaluator

# define a evaluator using `iob2` as tagging scheme
evaluator = SeqEvalEvaluator(tagging_scheme="iob2")
metrics = evaluator.compute(documents=docs_brat, predicted_entities=pred_ents)

In [66]:
for metric, value in metrics.items():
    print(f"{metric}: {value}")

overall_precision: 0.7115384615384616
overall_recall: 0.6607142857142857
overall_f1-score: 0.6851851851851851
overall_support: 56
overall_acc: 0.9991802076807209
alcool_precision: 0.6153846153846154
alcool_recall: 0.4444444444444444
alcool_f1-score: 0.5161290322580646
alcool_support: 18
situation_precision: 0.5
situation_recall: 1.0
situation_f1-score: 0.6666666666666666
situation_support: 1
tabagisme_precision: 0.7567567567567568
tabagisme_recall: 0.7567567567567568
tabagisme_f1-score: 0.7567567567567567
tabagisme_support: 37


In [67]:
for metric, value in metrics.items():
    print(f"{metric}: {value}")

overall_precision: 0.7115384615384616
overall_recall: 0.6607142857142857
overall_f1-score: 0.6851851851851851
overall_support: 56
overall_acc: 0.9991802076807209
alcool_precision: 0.6153846153846154
alcool_recall: 0.4444444444444444
alcool_f1-score: 0.5161290322580646
alcool_support: 18
situation_precision: 0.5
situation_recall: 1.0
situation_f1-score: 0.6666666666666666
situation_support: 1
tabagisme_precision: 0.7567567567567568
tabagisme_recall: 0.7567567567567568
tabagisme_f1-score: 0.7567567567567567
tabagisme_support: 37


## PREPARATION ANNOTATION ALCOOL

In [182]:
df,docs = extraction_finale("clinical_case2",option_melange=True)
print("fin")

fin


In [183]:
df['alcool'].value_counts()

alcool
UNKNOWN           685
ALCOOLIQUE         22
NON-ALCOOLIQUE     10
Name: count, dtype: int64

In [189]:
df_non_alcool = df[df["alcool"] == "NON-ALCOOLIQUE"]
df_alcool = df[df["alcool"] == "ALCOOLIQUE"].head(21)
df_unknown = df[df["alcool"] == "UNKNOWN"].head(20)

In [190]:
df_statut_alcool = pd.concat([df_non_alcool, df_alcool, df_unknown], ignore_index=True)

In [191]:
df_statut_alcool['alcool'].value_counts()

alcool
ALCOOLIQUE        21
UNKNOWN           20
NON-ALCOOLIQUE    10
Name: count, dtype: int64

In [192]:
df_statut_alcool.shape

(51, 5)

In [197]:
# Récupération dans un dossier des fichiers textes à annoter
import os
import shutil

chemin_dossier_source = "clinical_case2"
#chemin_dossier_destination = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_alcool"
# On crée le dossier destination si il n'existe pas 
os.makedirs(chemin_dossier_destination, exist_ok=True)

for nom_fichier in df_statut_alcool['nom fichier']:
    chemin_fichier_source = os.path.join(chemin_dossier_source, nom_fichier)
    chemin_fichier_destination = os.path.join(chemin_dossier_destination, nom_fichier)
    shutil.copy(chemin_fichier_source, chemin_fichier_destination)

In [31]:
#On remplace les fichiers deja annoté présent dans le dossier tabac vers dossier alcool
import os
import shutil

dossier_alcool = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_alcool"
dossier_tabac = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_tabac"

# Liste des fichiers se terminant par "ann" du dossier alcool
fichiers_alcool = [f for f in os.listdir(dossier_alcool) if f.endswith("ann")]

for fichier_alcool in fichiers_alcool:
    chemin_fichier_alcool = os.path.join(dossier_alcool, fichier_alcool)
    chemin_fichier_tabac = os.path.join(dossier_tabac, fichier_alcool)

    # Vérifier si le fichier correspondant existe dans le dossier tabac
    if os.path.exists(chemin_fichier_tabac):
        os.remove(chemin_fichier_alcool)  # Supprimer le fichier du dossier alcool
        shutil.copy2(chemin_fichier_tabac, dossier_alcool)  # Copier le fichier du dossier tabac vers le dossier alcool
print("fin")

fin


In [202]:
# On tri le 
df_statut_alcool = df_statut_alcool.sort_values('nom fichier')

In [203]:
df_statut_alcool

nom fichier   
3      filehtml-23-cas.txt  \
2      filehtml-24-cas.txt   
9      filepdf-109-cas.txt   
35     filepdf-131-cas.txt   
41   filepdf-140-2-cas.txt   
38      filepdf-18-cas.txt   
44    filepdf-19-1-cas.txt   
49     filepdf-231-cas.txt   
13      filepdf-24-cas.txt   
11     filepdf-254-cas.txt   
0      filepdf-255-cas.txt   
21   filepdf-265-1-cas.txt   
31   filepdf-269-1-cas.txt   
42     filepdf-283-cas.txt   
10     filepdf-290-cas.txt   
39   filepdf-292-1-cas.txt   
43       filepdf-3-cas.txt   
25     filepdf-387-cas.txt   
14   filepdf-393-2-cas.txt   
46     filepdf-395-cas.txt   
17     filepdf-441-cas.txt   
16     filepdf-447-cas.txt   
29     filepdf-454-cas.txt   
20     filepdf-490-cas.txt   
12     filepdf-500-cas.txt   
23     filepdf-503-cas.txt   
4    filepdf-508-3-cas.txt   
40     filepdf-515-cas.txt   
22     filepdf-527-cas.txt   
26   filepdf-533-2-cas.txt   
24   filepdf-533-5-cas.txt   
28  filepdf-534-12-cas.txt   
19     filepdf-549-cas.txt   
50   filepdf-554-3-cas.txt   
32     filepdf-6-5-cas.txt   
15   filepdf-630-2-cas.txt   
27     filepdf-646-cas.txt   
5      filepdf-680-cas.txt   
48     filepdf-681-cas.txt   
8       filepdf-69-cas.txt   
45    filepdf-71-1-cas.txt   
1      filepdf-710-cas.txt   
6      filepdf-750-cas.txt   
7      filepdf-756-cas.txt   
18     filepdf-757-cas.txt   
33     filepdf-794-cas.txt   
36   filepdf-798-1-cas.txt   
47   filepdf-800-1-cas.txt   
34   filepdf-801-1-cas.txt   
37   filepdf-865-2-cas.txt   
30     filepdf-881-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## On fait tourner le code sur ces fichiers uniquement 

In [47]:
import os
import shutil

#dossier_source = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_alcool"
#dossier_destination = "51_fichiers_annotation_alcool"

# Liste des fichiers avec l'extension ".txt" dans le dossier source
fichiers_txt = [f for f in os.listdir(dossier_source) if f.endswith(".txt")]

for fichier_txt in fichiers_txt:
    chemin_fichier_source = os.path.join(dossier_source, fichier_txt)
    chemin_fichier_destination = os.path.join(dossier_destination, fichier_txt)
    shutil.copy2(chemin_fichier_source, chemin_fichier_destination)
print("fin")

fin


In [48]:
df,docs_medkit = extraction_finale("51_fichiers_annotation_alcool",option_melange=False)
print("fin")

fin


## EVALUATION: COMPARAISON ENTRE ANNOTATION BRAT ET MEDKIT

In [52]:
# On récupère les annotations brat

from medkit.io.brat import BratInputConverter

# Define Input Converter 
brat_converter = BratInputConverter()

path= "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_alcool"

# Load brat into a list of documents
docs_brat = brat_converter.load(dir_path = path)
len(docs_brat)

51

In [53]:
len(docs_brat)

51

In [54]:
pred_ents = convert_to_pred_ents(docs_medkit)
pred = convert_to_pred_ents(docs_brat)

In [55]:
pred_ents[10:15]

[[Entity(uid='6fadd2e2-197d-11ee-875d-b916f667b06e', label='alcool', attrs=EntityAttributeContainer(ann_id='6fadd2e2-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=3353, end=3360)], text='alcoolo')],
 [Entity(uid='6fadd2e3-197d-11ee-875d-b916f667b06e', label='alcool', attrs=EntityAttributeContainer(ann_id='6fadd2e3-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=430, end=437)], text='alcoolo')],
 [],
 [],
 [Entity(uid='6fadd2e4-197d-11ee-875d-b916f667b06e', label='alcool', attrs=EntityAttributeContainer(ann_id='6fadd2e4-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=43, end=53)], text='alcoolisme')]]

In [56]:
pred[10:15]

[[],
 [],
 [],
 [],
 [Entity(uid='6fadd31b-197d-11ee-875d-b916f667b06e', label='alcool', attrs=EntityAttributeContainer(ann_id='6fadd31b-197d-11ee-875d-b916f667b06e', attrs=[]), metadata={}, keys=set(), spans=[Span(start=43, end=53)], text='alcoolisme')]]

In [57]:
from medkit.core.text import TextDocument, Entity, Span
from medkit.text.metrics.ner import SeqEvalEvaluator

# define a evaluator using `iob2` as tagging scheme
evaluator = SeqEvalEvaluator(tagging_scheme="iob2")
metrics = evaluator.compute(documents=docs_brat, predicted_entities=pred_ents)

In [58]:
for metric, value in metrics.items():
    print(f"{metric}: {value}")

overall_precision: 0.5636363636363636
overall_recall: 0.4696969696969697
overall_f1-score: 0.5123966942148761
overall_support: 66
overall_acc: 0.9979125850461298
alcool_precision: 0.5405405405405406
alcool_recall: 0.40816326530612246
alcool_f1-score: 0.46511627906976744
alcool_support: 49
situation_precision: 0.5
situation_recall: 1.0
situation_f1-score: 0.6666666666666666
situation_support: 3
tabagisme_precision: 0.6666666666666666
tabagisme_recall: 0.5714285714285714
tabagisme_f1-score: 0.6153846153846153
tabagisme_support: 14


## PREPARATION ANNOTATION SITUATION FAMILIALE

In [68]:
df,docs = extraction_finale("clinical_case2",option_melange=True)
print("fin")

fin


In [70]:
df['situation'].value_counts()

situation
UNKNOWN     696
PAS SEUL     11
SEUL         10
Name: count, dtype: int64

In [87]:
df_seul = df[df["situation"] == "SEUL"].head(10)
df_pas_seul = df[df["situation"] == "PAS SEUL"].head(11)
df_unknown = df[df["situation"] == "UNKNOWN"].head(30)

In [89]:
df_statut_familial = pd.concat([df_seul, df_pas_seul, df_unknown], ignore_index=True)

In [76]:
df_statut_familial['situation'].value_counts()

situation
UNKNOWN     30
PAS SEUL    11
SEUL        10
Name: count, dtype: int64

In [77]:
df_statut_familial.shape

(51, 5)

In [80]:
# Récupération dans un dossier des fichiers textes à annoter
import os
import shutil

chemin_dossier_source = "clinical_case2"
chemin_dossier_destination = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_situation"
# On crée le dossier destination si il n'existe pas 
os.makedirs(chemin_dossier_destination, exist_ok=True)

for nom_fichier in df_statut_familial['nom fichier']:
    chemin_fichier_source = os.path.join(chemin_dossier_source, nom_fichier)
    chemin_fichier_destination = os.path.join(chemin_dossier_destination, nom_fichier)
    shutil.copy(chemin_fichier_source, chemin_fichier_destination)
print("fin")

fin


In [83]:
#On remplace les fichiers deja annoté présent dans le dossier tabac vers dossier alcool
import os
import shutil

dossier_a_modifier = "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_situation"
dossiers_ref = ["/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_tabac", 
               "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_alcool"]

# Liste des fichiers se terminant par "ann" du dossier a modifier
fichiers_a_modifier = [f for f in os.listdir(dossier_a_modifier) if f.endswith("ann")]

for dossier_ref in dossiers_ref:
    for fichier_a_modifier in fichiers_a_modifier:
        chemin_fichier_a_modifier = os.path.join(dossier_a_modifier, fichier_a_modifier)
        chemin_fichier_ref = os.path.join(dossier_ref, fichier_a_modifier)
        # Vérifier si le fichier correspondant existe dans le dossier ref
        if os.path.exists(chemin_fichier_ref):
            print(fichier_a_modifier)
            os.remove(chemin_fichier_a_modifier)  # Supprimer le fichier du dossier a modifier
            shutil.copy2(chemin_fichier_ref, dossier_a_modifier)  # Copier le fichier du dossier ref vers le dossier a modifier
print("fin")

filepdf-263-3-cas.ann
filepdf-88-cas.ann
filepdf-69-cas.ann
filepdf-750-cas.ann
filepdf-549-cas.ann
filepdf-881-cas.ann
filepdf-69-cas.ann
filepdf-750-cas.ann
filepdf-19-1-cas.ann
filepdf-801-1-cas.ann
fin


In [92]:
# On tri le 
df_statut_familial = df_statut_familial.sort_values('nom fichier').reset_index(drop=True)

In [93]:
df_statut_familial

,nom fichier,cas clinique,tabagisme,alcool,situation
0,filepdf-108-cas.txt,"Le patient, age de 48 ans, a ete admis pour une douleur lombaire droite presente depuis 12 heures et d'apparition spontanee. La douleur n'a pas cede aux antalgiques, ni aux anti-spasmodiques. Il n'y a aucune notion de traumatisme. A l'examen physique, il existait une sensibilite au niveau du point costo-musculaire droit. La tension arterielle etait de 180/120 mm Hg et la frequence cardiaque a 100/mn. Le reste de l'examen somatique etait normal. L'hemoglobine est de 12.1 g/dl et l'hematocrite de 30 %. La fonction renale etait normale. L'examen cytobacteriologique des urines mettait en evidence une hematurie microscopique. La radiographie de l'abdomen montre un elargissement de l'ombre renale droite et l'echographie ne permet pas de visualiser precisement les contours du rein. La tomodensitometrie abdominale montrait l'image d'une hemorragie peri-renale et la presence d'une masse au niveau du pole superieur droit, ne captant pas le produit de contraste (Figure 1). L'arteriographie selective montrait une zone hypo-vascularisee au sommet superieur du rein, mais l'embolisation a ete infructueuse (Figure 2). Le diagnostic d'hemorragie peri-renale spontanee etait retenu chez ce patient, mais l'origine futt a determiner. Apres 24 heures de surveillance, le taux d'hemoglobine etait tombe a 7,6 g/dl et l'hematocrite a 22%. Nous decidions de realiser une nephrectomie radicale par voie thoraco-lombaire. Celle-ci a permis de drainer plus de deux litres de sang. Les suites postoperatoires furent simples. L'examen anatomo-pathologique de la piece operatoire a montre la presence d'une formation blanchatre d'environ 2 cm de diametre au niveau du pole superieur du rein. L'examen microscopique montrait une proliferation de cellules a disposition papillaire, faites d'axes conjonctifs greles (Figure 3). Les cellules y possedaient un cytoplasme eosinophile assez abondant et leur activite mitotique etait nulle. Cette tumeur correspondait a un adenome papillaire. L'evolution post-operatoires a ete favorable avec un recul de 15 mois. Le patient se porte bien avec une fonction renale normale et une tension arterielle equilibree.",UNKNOWN,UNKNOWN,UNKNOWN
1,filepdf-112-cas.txt,"Mr. H. M... age de 45 ans, ouvrier, ayant des antecedents de chancre syphilitique, etait admis dans le service pour un syndrome obstructif du bas appareil urinaire. L'histoire de la maladie remontait a 10 ans par l'installation progressive d'une dysurie, pollakiurie (2 a 3 fois/nuit) et d'une incontinence urinaire par regorgement. Sa symptomatologie s'etait aggravee par l'apparition d'une boiterie et d'une deformation indolore du genou droit . L'examen clinique revelait une deformation et une augmentation du volume du genou droit, un globe vesical et une fuite d'urine a la palpation de la region hypogastrique. Le toucher rectal montrait une prostate induree dans sa totalite et de petit volume. L'examen neurologique montrait l'absence de deficit moteur, une hypotonie et une depression des reflexes osteo-tendineux aux membres inferieurs, une hypo-esthesie du gland et un reflexe photo-moteur diminue des deux cotes avec un myosis serre a gauche (signe d'Argyll-Robertson). L'ensemble constitue le syndrome cordonal posterieur. La fonction renale etait normale et l'ECBU etait sterile. La serologie syphilitique etait fortement positive et la ponction lombaire avait revele une meningite lymphocytaire. L'echographie renale etait normale L'uretrocystographie retrograde et mictionnelle avait montre une vessie de grande capacite avec un important residu post mictionnel (d'environ 500 ml) (Figure 1). La cystomanometrie et le profil uretral objectivaient une vessie hypo-active avec une capacite vesicale de 1200 ml et une pression uretrale normale. Le debit maximum etait de 10 ml/ seconde. La radiographie du genou droit montrait une destruction articulaire avec luxation de l'articulation femoro-tibiale (Fig.2). Le patient

## On fait tourner le code sur ces fichiers uniquement 

In [94]:
df,docs_medkit = extraction_finale("51_fichiers_annotation_situation",option_melange=False)
print("fin")

fin


## EVALUATION: COMPARAISON ENTRE ANNOTATION BRAT ET MEDKIT

In [95]:
# On récupère les annotations brat

from medkit.io.brat import BratInputConverter

# Define Input Converter 
brat_converter = BratInputConverter()

path= "/home/mhassani/Documents/Stage/env/brat-master/data/51_fichiers_annotation_situation"

# Load brat into a list of documents
docs_brat = brat_converter.load(dir_path = path)
len(docs_brat)

51

In [96]:
pred_ents = convert_to_pred_ents(docs_medkit)
pred = convert_to_pred_ents(docs_brat)

In [97]:
from medkit.core.text import TextDocument, Entity, Span
from medkit.text.metrics.ner import SeqEvalEvaluator

# define a evaluator using `iob2` as tagging scheme
evaluator = SeqEvalEvaluator(tagging_scheme="iob2")
metrics = evaluator.compute(documents=docs_brat, predicted_entities=pred_ents)

In [98]:
for metric, value in metrics.items():
    print(f"{metric}: {value}")

overall_precision: 0.7142857142857143
overall_recall: 0.625
overall_f1-score: 0.6666666666666666
overall_support: 40
overall_acc: 0.9988387880520223
alcool_precision: 0.5
alcool_recall: 0.1875
alcool_f1-score: 0.2727272727272727
alcool_support: 16
situation_precision: 0.7391304347826086
situation_recall: 0.9444444444444444
situation_f1-score: 0.8292682926829269
situation_support: 18
tabagisme_precision: 0.8333333333333334
tabagisme_recall: 0.8333333333333334
tabagisme_f1-score: 0.8333333333333334
tabagisme_support: 6
